# <big><big><big><font color='#5D6D7E'>**ENTRENAMIENTO DE UNA RED NEURONAL PARA EMULACION DE UNA CURVA UTILIZANDO KERAS**</font></big><big><big>
https://www.tensorflow.org/tutorials/load_data/pandas_dataframe<br>
    
En este archivo trataremos de emular 1 curva a partir de algunas otras presentes en el mismo archivo.<br>
    
Los pasos son los siguientes:
    
    
1. [POZOS DE TRABAJO](#counter_curvas)
    - 1.1 [Revisamos todos los archivos .LAS en la carpeta de trabajo](#counter_curvas_1)
    - 1.2 [Elegimos los archivos .LAS de trabajo](#counter_curvas_2)
    - 1.3 [Exportamos los archivos .LAS con lasio y pasamos cada .LAS a 1 DF pandas](#counter_curvas_3)
    - 1.4 [Chequeamos cantidad de pozos y cantidad de veces que aparece cada curva en todos los pozos](#counter_curvas_4)<br><br><br>
2. [POZO ENTRENADOR](#PozoEntrenador)
    - 2.1 [Carga y QC de datos del pozo entrenador](#PozoEntrenador_LoadQc)
        - 2.1.1 [Cargamos con lasio el archivo .LAS con los datos del pozo entrenador y creamos un nuevo DF pandas](#PozoEntrenador_LasioPandas)
        - 2.1.2 [PLOT QC](#PozoEntrenador_PlotQc)
        - 2.1.3 [Declaración de variables (para plot combinado y para entenamiento de la red)](#PozoEntrenador_Declaracion)
        - 2.1.4 [PLOT COMBINADO](#PozoEntrenador_Combo)<br><br>
    - 2.2 [Entrenamiento de la red](#PozoEntrenador_Modelo)
        - 2.2.1 [Creacion df con las curvas que utilizaremos como features y la curva target, ambas en el tramo del pozo conveniente](#PozoEntrenador_Modelo_decl)
        - 2.2.2 [Creación de funciones para graficar el modelo](#PozoEntrenador_Modelo_defs)
        - 2.2.3 [Keras: Entrenamiento del modelo (fit)](#PozoEntrenador_Modelo_fit)
        - 2.2.4 [Keras: Aplicación del modelo - obtención de la curva emulada (predict)](#PozoEntrenador_Modelo_predict)
        - 2.2.5 [ Plot para comparar curva target vs. curva emulada](#PozoEntrenador_Modelo_predict_plot)

In [ ]:
import pandas as pd
import numpy as np
import lasio

from collections import Counter

import matplotlib.pyplot as plt
import mplcursors
%matplotlib inline
%matplotlib notebook
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import StrMethodFormatter

# import tensorflow as tf
# import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import fnmatch
import glob, os
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')    

<a id='counter_curvas'></a>
# <big><big><font color='#8E44AD'>1. POZOS DE TRABAJO</font></big></big>

<a id='counter_curvas_1'></a>
# <font color='#1F618D'>1.1 Revisamos todos los archivos .LAS en la carpeta de trabajo</font>

In [ ]:
path=input("Escriba el PATH completo de la carpeta que contiene los .CSV y presione INTRO.(Ejemplo: D:\Pablo\Python\working_files\pozos)")

clear_output(wait=True)
# D:\Pablo\Python\working_files\pozos
for file in os.listdir(path):
    if file.endswith(".las"):
        print(os.path.join(file))

<a id='counter_curvas_2'></a>
# <font color='#1F618D'>1.2 Elegimos los archivos .LAS de trabajo</font>

In [ ]:
if 'las_file_list' in locals():
    del las_file_list
else:
    las_file_list = []

las_file_list = []   
las_file = []


pozo_nomb_comienzo=input("Escriba las primeras letras/numeros de los Pozos con los que va a trabajar y presione INTRO. Si va a utilizar todos los .LAS de la carpeta, solo presione INTRO")

clear_output(wait=True)

for file in os.listdir(path):
    if file.endswith('.las') and file.startswith((pozo_nomb_comienzo.lower(),pozo_nomb_comienzo.upper(),pozo_nomb_comienzo.capitalize())):
        las_file_list.append(path+"\\"+file)
        las_file.append(file)
las_file

<a id='counter_curvas_3'></a>
# <font color='#1F618D'>1.3 Exportamos los archivos .LAS con lasio y pasamos cada .LAS a 1 DF pandas (Cada DataFrame hereda el nombre del archivo, sin caracteres especiales)</font>

In [ ]:
las_list = []

for i in range(0,len(las_file)):
         
    string = las_file[i][:-3]                 
    string = ''.join(e for e in string if e.isalnum()) 
    
    locals()[string] = lasio.read(las_file_list[i]).df()   
    
    las_list.append(string)

for elem in range(len(las_list)):
    locals()[las_list[elem]]['DEPTH'] = locals()[las_list[elem]].index


print ('\033[1m'+f'TOTAL DE DATAFRAMES: {len(las_list)}')
for elem in las_list:
    print(elem)

<a id='counter_curvas_4'></a>
# <font color='#1F618D'>1.4 <font color='#1F618D'>Chequeamos cantidad de pozos y cantidad de veces que aparece cada curva en todos los pozos</font></font>

In [ ]:
print(f'NUMERO DE POZOS:{len(las_list)}')
print(*las_list,sep=', ')

print('\n')
print('NOMBRE DE LA CURVA,','CANTIDAD DE POZOS DONDE APARECE:')

curves_list = []

# hago un merge de todas las curvas en una sola LIST
for i in range(len(las_list)):
    curves_list = curves_list + (list(locals()[las_list[(i)]].columns))
    
# ordeno alfabeticamente la LIST de curvas y luego encuentro las curvas más comunes en la LIST
curves_list.sort()
dict_curves = dict(Counter(curves_list).most_common())
Counter(curves_list).most_common()


<a id='PozoEntrenador'></a>
# <big><big><font color='#8E44AD'>2. POZO ENTRENADOR</font></big></big>

<a id='PozoEntrenador_LoadQc'></a>
# <font color='#1F618D'>2.1 Carga y QC de datos del pozo entrenador</font>

<a id='PozoEntrenador_LasioPandas'></a>
# <font color='#BA4A00'>2.1.1 Cargamos con lasio el archivo .LAS con los datos del pozo entrenador y creamos un nuevo DF pandas </font>

In [ ]:
las = lasio.read(r"D:\Pablo\Python\working_files\Pozos\well_13.las")

well = las.df()
well['DEPTH'] = well.index

# Ordenar alfabeticamente
well = well.reindex(sorted(well.columns), axis=1)

# Nombre del pozo 
for count, curve in enumerate(well):
    (count)
print ('\033[1m' + f"POZO: {las.well.WELL.value}","\n" + f"Total de curvas:{count+1}")
pd.set_option('display.max_columns', None)          # Muestra todas las columnas 
well

In [ ]:
### DATOS DE CURVAS DEL ARCHIVO LASIO ORDENADAS ALFBETICAMENTE ###

lasio_file = las


print (f"POZO: {lasio_file.well.WELL.value}","\t",f"YACIMIENTO: {lasio_file.well.FLD.value}""\n"
       f"TOP: {lasio_file.well.STRT.value:.2f} {lasio_file.well.STRT.unit}","\t"
       f"BOTTOM: {lasio_file.well.STOP['value']:.2f} {lasio_file.well.STOP['unit']}")

print(f"TOTAL DE CURVAS: {count+1}","(ORDENADAS ALFABETICAMENTE)","\n")

## Curvas       
for count, i in enumerate(range(len(lasio_file.curves))):
                          print(count+1,
                                lasio_file.curves[sorted(lasio_file.curves.keys())[i]].mnemonic,
                                ", Unit:", lasio_file.curves[sorted(lasio_file.curves.keys())[i]].unit,
                                ", Description:", lasio_file.curves[sorted(lasio_file.curves.keys())[i]].descr)    

<a id='PozoEntrenador_PlotQc'></a>
# <font color='#BA4A00'>2.1.2 PLOT QC</font>

Este PLOT está enfocado en la cantidad de pozos donde se registró cada curva, y en la búsqueda de gaps

In [ ]:
################## DATOS DE ENTRADA ##################

df = well            # Nombre del dataframe pandas que creamos
lasio_file = las      # Nombre del archivo lasio que creamos al principio   
  
top_plot_qc = lasio_file.well.STRT.value   # Top plot
bot_plot_qc = lasio_file.well.STOP.value   # Bottom plot

data_color = '#BD1E0E'     # color de relleno de los datos
data_alpha = 0.8           # transparencia del color (0: transparente 1:opaco)

####################### CODIGO #######################

if 'dict_curves' in locals():
    pass
else:
    dict_curves = {}

df_nan = df.notnull() * 1

fig = plt.subplots(figsize=((len(well.columns)*.8),(len(well.columns)*.7))) #Set up the plot axes
 
for i in range(len(well.columns)):
    ax = plt.subplot2grid((1,len(well.columns)), (0,i), rowspan=1, colspan = 1)
    ax.plot(df_nan.iloc[:,i], df_nan.index, lw=0)
    ax.set_ylim(bot_plot_qc, top_plot_qc)
    ax.set_xlim(0, 1)
    
    if dict_curves.get(df_nan.columns[i]) == None:
        ax.set_title(df_nan.columns[i][:4], rotation=0)         # titulo columnas: [:4] solo muetras primeras 4 letras
                                                            # titulo columnas: rotation=0 horizontal, rotation=90 vertical
    else:
        ax.set_title(label=f'{df_nan.columns[i][:4]}\n{dict_curves.get(df_nan.columns[i])}', rotation=0)
        
    ax.set_facecolor('#F2F4F4')
    ax.fill_betweenx(df_nan.index, 0, df_nan.iloc[:,i], facecolor= data_color, alpha= data_alpha)       
    plt.setp(ax.get_xticklabels(), visible = False)
    if i > 0:
        plt.setp(ax.get_yticklabels(), visible = False)
    else:    # Sets the label for the leftmost subplot
        ax.set_ylabel('DEPTH', fontsize=22,family='cursive')



plt.subplots_adjust(wspace=0)
plt.suptitle(t=f'\nPOZO ENTRENADOR: {lasio_file.well.WELL.value}\nCANTIDAD TOTAL DE POZOS: {len(las_list)}',
             size=25,ha="left", x=0.125, y=1.01, weight=900,family='cursive')

                                            ## The font.weight property has effectively 13 values: normal, bold,
                                            ## bolder, lighter, 100, 200, 300, ..., 1000.  Normal=400, bold=700
        
 
mplcursors.cursor(hover=True)
plt.show()

<a id='PozoEntrenador_Declaracion'></a>
# <font color='#BA4A00'>2.1.3 Declaración de variables para plot combinado</font>
https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe

- Si nos equivocamos en el nombre de alguna de las curvas de la DECLARACION DE VARIABLES PARA PLOT COMBINADO, y se crean curvas con valores **Nan**, o algo similar, podemos borrarlas y volver a crearlas con el nombre correcto corriendo de nuevo la celda de DECLARACION DE VARIABLES PARA PLOT COMBINADO.

In [ ]:
# Ver curvas del dataframe si vemos que el PLOT no graficó las curvas correctamente. 
# well.describe()

In [ ]:
# Borrar columnas si es necesario, corregir el nombre de las curvas en la celda de abajo (DECLARACION DE VARIABLES
# PARA PLOT COMBINADO), y volverla a correr. Luego correr la celda del PLOT COMBINADO para verificar las correcciones.

# well.drop('MDEP', inplace=True, axis=1)                           # Borrar 1 sola columna
# well.drop(['MDEP','MPHS','DCON','BIT'], inplace=True, axis=1)     # Borrar muliples columnas

In [ ]:
##################################################
#                                                #
#  DECLARACION DE VARIABLES PARA PLOT COMBINADO  #
#                                                #
##################################################


bitsize = 12.25

## ax1
crv_cal = 'CALI'             # CALIPER
crv_bit = 'BIT'             # BIT SIZE
crv_gr = 'GR'               # GAMMA RAY
crv_sp = 'SP'              # SPONTANEOUS POTENTIAL

# ax2
crv_dp_res = 'RDEP'         # DEEP RESISTIVITY
crv_sh_res = 'RMED'         # SHALLOW RESISTIVITY

#ax3
crv_neu_por = 'NPHI'        # NEUTRON POROSITY
crv_den_bulk = 'RHOB'       # BULK DENSITY
crv_nmr_por = 'EMPTYPOR'        # OTHER POROSITY

#ax4
crv_con = 'DCON'            # CONDUCTIVITY


##############################################################
if crv_cal in well.columns:
    pass
else:
    well[crv_cal]=np.nan

##############    

if crv_bit in well.columns:
    pass
else:
    well[crv_bit]=bitsize

##############    

if crv_gr in well.columns:
    pass
else:
    well[crv_gr]=np.nan
    
##############  

if crv_sp in well.columns:
    pass
else:
    well[crv_sp]=np.nan
    
##############  

if crv_dp_res in well.columns:
    pass
else:
    well[crv_dp_res]=np.nan
    
##############  

if crv_sh_res in well.columns:
    pass
else:
    well[crv_sh_res]=np.nan
    
############## 

if crv_neu_por in well.columns:
    pass
else:
    well[crv_neu_por]=np.nan

############## 

if crv_nmr_por in well.columns:
    pass
else:
    well[crv_nmr_por]=np.nan

############## 

if crv_den_bulk in well.columns:
    pass
else:
    well[crv_den_bulk]=np.nan
    
##############      
    
if crv_con in well.columns:
    pass
else:
    well[crv_con]=1000/well[crv_dp_res]    
        
################

<a id='PozoEntrenador_Combo'></a>
# <font color='#BA4A00'>2.1.4 PLOT COMBINADO</font>
https://engineeringfordatascience.com/posts/matplotlib_subplots/

In [ ]:
top_plot= las.well.STRT.value
bot_plot= las.well.STOP.value

## numero de lineas verticales del track que va a depender de la escala de la curva
ax1_ticks = 6         # Convencionales
ax2_ticks = 6         # Resistividades
ax3_ticks = 7         # Porosidades
ax4_ticks = 6         # Conductividad

fig, ax = plt.subplots(figsize=(20,20))#Set up the plot axes
   
ax1 = plt.subplot2grid((1,6), (0,0), rowspan=1, colspan = 1) 
ax11 = ax1.twiny()
ax12 = ax1.twiny()

ax2 = plt.subplot2grid((1,6), (0,1), rowspan=1, colspan = 1, sharey = ax1)
ax21 = ax2.twiny()

ax3 = plt.subplot2grid((1,6), (0,2), rowspan=1, colspan = 1, sharey = ax1) #
ax31 = ax3.twiny() 
ax32 = ax3.twiny() 

ax4 = plt.subplot2grid((1,6), (0,3), rowspan=1, colspan = 1, sharey = ax1) #

################################################################################

### CAL ###
cal_izq = 10     # min=izq 
cal_der = 20     # max=der

ax1.plot(well[crv_cal], well.index, color = "black", linewidth = 0.5)
ax1.set_ylabel("Depth (m)")

if abs(top_plot-bot_plot) > 500:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/99)))
    
elif abs(top_plot-bot_plot) <= 500 and abs(top_plot-bot_plot) > 200:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/49)))
else:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/24)))
    
ax1.set_xlabel(f"Caliper {well[crv_cal].name}")
ax1.set_xlim(cal_izq, cal_der)
ax1.xaxis.label.set_color("black")
ax1.tick_params(axis='x', colors="black")
ax1.spines["top"].set_edgecolor("black")
ax1.set_xticks(np.arange(cal_izq, cal_der+1, ((cal_der-cal_izq)/(ax1_ticks-1))))
ax1.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))

ax1.fill_betweenx(well.index, well[crv_bit], well[crv_cal],
                  where=well[crv_bit]>=well[crv_cal], interpolate=True, color='brown')
ax1.fill_betweenx(well.index, well[crv_bit], well[crv_cal],
                  where=well[crv_bit]<=well[crv_cal], interpolate=True, color='#a6ecfa')
################################################################################

### GR ###
gr_izq=0        
gr_der=200      

ax11.plot(well[crv_gr], well.index, color = "green", linewidth = 1.5)
ax11.set_xlabel(f"Gamma Ray ({well[crv_gr].name})")
ax11.xaxis.label.set_color("green")
ax11.set_xlim(gr_izq, gr_der)
ax11.tick_params(axis='x', colors="green")
ax11.spines["top"].set_position(("axes", 1.12))
ax11.spines["top"].set_visible(True)
ax11.spines["top"].set_edgecolor("green")
ax11.title.set_color('green')
ax11.set_xticks(np.arange(gr_izq, gr_der+1, ((gr_der-gr_izq)/(ax1_ticks-1))))
ax11.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### SP ###
sp_izq=-80   
sp_der=20     

ax12.plot(well[crv_sp], well.index, color = "red", linewidth = 1.5)
ax12.set_xlabel(f"SP ({well[crv_sp].name})")
ax12.xaxis.label.set_color("red")
ax12.set_xlim(sp_izq,sp_der)
ax12.tick_params(axis='x', colors="r")
ax12.spines["top"].set_position(("axes", 1.07))
ax12.spines["top"].set_visible(True)
ax12.spines["top"].set_edgecolor("r")
ax12.title.set_color('r')
ax12.set_xticks(np.arange(sp_izq, sp_der+1, ((sp_der-sp_izq)/(ax1_ticks-1))))
ax12.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### RESD ###
res_izq=0.1     
res_der=100     

ax2.plot(well[crv_dp_res], well.index, color = "blue", linewidth = 2)
ax2.set_xlabel(f"Deep Resistivity ({well[crv_dp_res].name})")
ax2.set_xlim(res_izq, res_der)
ax2.xaxis.label.set_color("blue")
ax2.tick_params(axis='x', colors="blue")
ax2.spines["top"].set_edgecolor("blue")
ax2.set_xticks(np.arange(res_izq, res_der+.1, (res_der/(ax2_ticks-1))))
ax2.set_xscale("log")
ax2.grid(True, which="both", axis='x')
ax2.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### RESS ###


ax21.plot(well[crv_sh_res], well.index, color = "r", linewidth = 0.5)
ax21.set_xlabel(f"Shallow Resistivity ({well[crv_sh_res].name})")
ax21.set_xlim(res_izq, res_der)
ax21.xaxis.label.set_color("r")
ax21.spines["top"].set_position(("axes", 1.07))
ax21.spines["top"].set_visible(True)
ax21.tick_params(axis='x', colors="r")
ax21.spines["top"].set_edgecolor("r")
ax21.set_xticks(np.arange(res_izq, res_der+.1, ((res_der-res_izq)/(ax2_ticks-1))))
ax21.set_xscale("log")
ax21.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### DEN BULK ###
denbulk_izq = 1.65      
denbulk_der = 2.65      


ax3.plot(well[crv_den_bulk], well.index, color = "red", linewidth = 1.0)
ax3.set_xlabel(f"Density Bulk ({well[crv_den_bulk].name})")
ax3.set_xlim(denbulk_izq,denbulk_der)
ax3.xaxis.label.set_color("red")
ax3.tick_params(axis='x', colors="r")
ax3.spines["top"].set_edgecolor("r")
ax3.set_xticks(np.arange(denbulk_der, denbulk_izq+.01, ((denbulk_izq-denbulk_der)/(ax3_ticks-1))))
ax3.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NEU POR ###
neupor_izq = .6     
neupor_der = 0      

ax31.plot(well[crv_neu_por], well.index, color = "blue", linewidth = 1.0)
ax31.set_xlabel(f"Neutron Porosity ({well[crv_neu_por].name})")
ax31.xaxis.label.set_color("b")
ax31.set_xlim(neupor_izq,neupor_der)
ax31.tick_params(axis='x', colors="blue")
ax31.spines["top"].set_position(("axes", 1.07))
ax31.spines["top"].set_visible(True)
ax31.spines["top"].set_edgecolor("blue")
ax31.set_xticks(np.arange(neupor_der, neupor_izq+.01, (abs(neupor_der-neupor_izq)/(ax3_ticks-1))))
ax31.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### AUX POR ###
auxpor_izq = .6    
auxpor_der = 0     
auxpor_desc = 'Empty Porosity'

ax32.plot(well[crv_nmr_por], well.index, color = "black", linewidth = 1.0)
ax32.set_xlabel(f"{auxpor_desc} ({well[crv_nmr_por].name})")
ax32.xaxis.label.set_color("black")
ax32.set_xlim(auxpor_izq,auxpor_der)
ax32.tick_params(axis='x', colors="black")
ax32.spines["top"].set_position(("axes", 1.12))
ax32.spines["top"].set_visible(True)
ax32.spines["top"].set_edgecolor("black")
ax32.set_xticks(np.arange(auxpor_der, auxpor_izq+.01, (abs(auxpor_der-auxpor_izq)/(ax3_ticks-1))))
ax32.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################


### CONDUCTIVITY ###
cond_izq = 2000     
cond_der = 0        

ax4.plot(well[crv_con], well.index, color = "black", linewidth = 1.5)
ax4.set_xlabel(f"Conductivity ({well[crv_con].name})")
ax4.set_xlim(cond_izq,cond_der)
ax4.xaxis.label.set_color("black")
ax4.tick_params(axis='x', colors="black")
ax4.spines["top"].set_edgecolor("black")
ax4.set_xticks(np.arange(cond_der, cond_izq+1, (abs(cond_der-cond_izq)/(ax4_ticks-1))))
ax4.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################


fig.suptitle(las.well.WELL.value+"\n"+"YACIMIENTO: "+las.well.FLD.value+"\n"+"COMPAÑIA: "+las.well.COMP.value,size=26,
             x=0.36,y=1)

# Common functions for setting up the plot can be extracted into
# a for loop. This saves repeating code.

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylim(bot_plot, top_plot)
    ax.grid(which='major', color='lightgrey', linestyle='-')
    ax.xaxis.set_ticks_position("top")
    ax.xaxis.set_label_position("top")
    ax.spines["top"].set_position(("axes", 1.02))
 
    
for ax in [ax2, ax3, ax4]:
    plt.setp(ax.get_yticklabels(), visible = False)
  

fig.subplots_adjust(wspace = 0.2)

mplcursors.cursor(hover=True)
plt.tight_layout()
plt.show()


<a id='PozoEntrenador_Modelo'></a>
# <font color='#1F618D'>2.2 Entrenamiento de la red</font>

<a id='PozoEntrenador_Modelo_decl'></a>
# <font color='#BA4A00'>2.2.1 Creacion df con las curvas que utilizaremos como features y la curva target, ambas en el tramo del pozo conveniente</font>

In [ ]:
# DECLARACION DE CURVAS FEATURES y TARGET

##############################################################
# TRAMO A PROCESAR

well_top = 1140
well_bot = 2990


#######################################
# CURVAS PARA ENTRENAMIENTO CON KERAS

crv_target = 'NPHI'

crv_feature1 = 'CALI'
crv_feature2 = 'GR'
crv_feature3 = 'DTC'
crv_feature4 = 'PEF'
crv_feature5 = 'RHOB'
crv_feature6 = 'RDEP'
crv_feature7 = 'RMED'
crv_feature8 = 'SP'
crv_feature9 = 'NADA'
crv_feature10 = 'NADA'


##############################################################

well_train = well[(well.DEPTH >= well_top) & (well.DEPTH <= well_bot)]

well_train['target'] = well_train[crv_target]

if 'features_list' in locals():
    del features_list
else:
    features_list = []

features_list = []    

### 

if crv_feature1 in well_train.columns:
    features_list.append(crv_feature1)
else:
    pass

if crv_feature2 in well_train.columns:
    features_list.append(crv_feature2)
else:
    pass

if crv_feature3 in well_train.columns:
    features_list.append(crv_feature3)
else:
    pass

if crv_feature4 in well_train.columns:
    features_list.append(crv_feature4)
else:
    pass

if crv_feature5 in well_train.columns:
    features_list.append(crv_feature5)
else:
    pass

if crv_feature6 in well_train.columns:
    features_list.append(crv_feature6)
else:
    pass

if crv_feature7 in well_train.columns:
    features_list.append(crv_feature7)
else:
    pass

if crv_feature8 in well_train.columns:
    features_list.append(crv_feature8)
else:
    pass

if crv_feature9 in well_train.columns:
    features_list.append(crv_feature9)
else:
    pass

if crv_feature10 in well_train.columns:
    features_list.append(crv_feature10)
else:
    pass

well_train_features = well_train[features_list]
well_train_features

# <big><big><big><font color='#y67E22'>**KERAS**</font></big></big></big>
**input_dim:** https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-find-the-value-for-keras-input_shape-input_dim.md<br>


<a id='PozoEntrenador_Modelo_defs'></a>
# <big><font color='#D35400'>2.2.2 Creación de funciones para graficar el modelo</font></big>

In [ ]:
def get_plot_no_val(historia):
        
    plt.rcParams["figure.figsize"] = (14,3.5)
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.plot(historia.history[f'{list(historia.history.keys())[0]}'])
    #plt.plot(historia.history[f'{list(historia.history.keys())[2]}'])
    plt.ylabel(f'Pérdida ({list(historia.history.keys())[0]})')
    plt.xlabel('Iteración')
 #   plt.legend(['Entrenamiento','Validación'])

    plt.subplot(1,2,2)
    plt.plot(historia.history[f'{list(historia.history.keys())[1]}'])
   # plt.plot(historia.history[f'{list(historia.history.keys())[3]}'])
    plt.ylabel(f'Métrica ({list(historia.history.keys())[1]})')
    plt.xlabel('Iteración')
#    plt.legend(['Entrenamiento','Validación'])

    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    
    plt.show()
    
##############################    
    
def get_plot_val(historia):
    
    
    plt.rcParams["figure.figsize"] = (14,3.5)
    plt.subplots_adjust(wspace=0.5)
    plt.subplot(1,2,1)
    plt.plot(historia.history[f'{list(historia.history.keys())[0]}'])
    plt.plot(historia.history[f'{list(historia.history.keys())[2]}'])
    plt.ylabel(f'Pérdida ({list(historia.history.keys())[0]})')
    plt.xlabel('Iteración')
    plt.legend(['Entrenamiento','Validación'])

    plt.subplot(1,2,2)
    plt.plot(historia.history[f'{list(historia.history.keys())[1]}'])
    plt.plot(historia.history[f'{list(historia.history.keys())[3]}'])
    plt.ylabel(f'Métrica ({list(historia.history.keys())[1]})')
    plt.xlabel('Iteración')
    plt.legend(['Entrenamiento','Validación'])

    ax = plt.gca()
    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    
    plt.show()

<a id='PozoEntrenador_Modelo_fit'></a>
# <big><font color='#D35400'>2.2.3 Keras: Entrenamiento del modelo (fit)</font></big>

In [ ]:
X = well_train_features
Y = well_train['target']

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=True)


model = Sequential()
#model.add(Dense(1000, input_dim=len(well_train_features.columns), activation='relu'))
model.add(Dense(1000, activation='relu'))
# model.add(Dropout(0.9))
model.add(Dense(250, activation='relu'))
# model.add(Dropout(0.8))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['MeanSquaredError'])

# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, update_freq='batch')

historia = model.fit(x_train, y_train, batch_size=10, epochs=10, validation_data=(x_test,y_test))

clear_output()

get_plot_val(historia) 

<a id='PozoEntrenador_Modelo_predict'></a>
# <big><font color='#D35400'>2.2.4 Keras: Aplicación del modelo - obtención de la curva emulada (predict)</font></big>

In [ ]:
crv_emulated = 'NPHI_emulada'
well_train[crv_emulated]=model.predict(
    X,
    batch_size=10,
    verbose=0,
    steps=None,
    callbacks=None,
    max_queue_size=10,
    workers=1,
    use_multiprocessing=False)

well_train

<a id='PozoEntrenador_Modelo_predict_plot'></a>
# <big><font color='#D35400'>2.2.5 Plot para comparar curva target vs. curva emulada</font></big>

In [ ]:
top_plot= well_top
bot_plot= well_bot

crv_target = 'NPHI'
crv_emulated = 'NPHI_emulada'

# Escalas curvas
target_izq = .6       
target_der = 0    



# numero de lineas verticales del track que va a depender de la escala de la curva
ax1_ticks = 6         # Convencionales
ax2_ticks = 6         # Resistividades
ax3_ticks = 7         # Porosidades
ax4_ticks = 7         # Target & Emulated



##################### CODIGO ###################################################

fig, ax = plt.subplots(figsize=(20,20))#Set up the plot axes
   
ax1 = plt.subplot2grid((1,6), (0,0), rowspan=1, colspan = 1) 
ax11 = ax1.twiny()
ax12 = ax1.twiny()

ax2 = plt.subplot2grid((1,6), (0,1), rowspan=1, colspan = 1, sharey = ax1)
ax21 = ax2.twiny()

ax3 = plt.subplot2grid((1,6), (0,2), rowspan=1, colspan = 1, sharey = ax1) #
ax31 = ax3.twiny() 
ax32 = ax3.twiny() 

ax4 = plt.subplot2grid((1,6), (0,3), rowspan=1, colspan = 1, sharey = ax1) #
ax41 = ax4.twiny() 
################################################################################

### CAL ###
cal_izq = 10     # min=izq 
cal_der = 20     # max=der

ax1.plot(well_train[crv_cal], well_train.index, color = "black", linewidth = 0.5)
ax1.set_ylabel("Depth (m)")

if abs(top_plot-bot_plot) > 500:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/99)))
    
elif abs(top_plot-bot_plot) <= 500 and abs(top_plot-bot_plot) > 200:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/49)))
else:
    ax1.yaxis.set_major_locator(MaxNLocator(round(abs(top_plot-bot_plot)/24)))
    
ax1.set_xlabel(f"Caliper {well_train[crv_cal].name}")
ax1.set_xlim(cal_izq, cal_der)
ax1.xaxis.label.set_color("black")
ax1.tick_params(axis='x', colors="black")
ax1.spines["top"].set_edgecolor("black")
ax1.set_xticks(np.arange(cal_izq, cal_der+1, ((cal_der-cal_izq)/(ax1_ticks-1))))
ax1.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))

ax1.fill_betweenx(well_train.index, well_train[crv_bit], well_train[crv_cal],
                  where=well_train[crv_bit]>=well_train[crv_cal], interpolate=True, color='brown')
ax1.fill_betweenx(well_train.index, well_train[crv_bit], well_train[crv_cal],
                  where=well_train[crv_bit]<=well_train[crv_cal], interpolate=True, color='#a6ecfa')
################################################################################

### GR ###
gr_izq=0        
gr_der=200      

ax11.plot(well_train[crv_gr], well_train.index, color = "green", linewidth = 1.5)
ax11.set_xlabel(f"Gamma Ray ({well_train[crv_gr].name})")
ax11.xaxis.label.set_color("green")
ax11.set_xlim(gr_izq, gr_der)
ax11.tick_params(axis='x', colors="green")
ax11.spines["top"].set_position(("axes", 1.1))
ax11.spines["top"].set_visible(True)
ax11.spines["top"].set_edgecolor("green")
ax11.title.set_color('green')
ax11.set_xticks(np.arange(gr_izq, gr_der+1, ((gr_der-gr_izq)/(ax1_ticks-1))))
ax11.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### SP ###
sp_izq=-80   
sp_der=20     

ax12.plot(well_train[crv_sp], well_train.index, color = "red", linewidth = 1.5)
ax12.set_xlabel(f"SP ({well_train[crv_sp].name})")
ax12.xaxis.label.set_color("red")
ax12.set_xlim(sp_izq,sp_der)
ax12.tick_params(axis='x', colors="r")
ax12.spines["top"].set_position(("axes", 1.06))
ax12.spines["top"].set_visible(True)
ax12.spines["top"].set_edgecolor("r")
ax12.title.set_color('r')
ax12.set_xticks(np.arange(sp_izq, sp_der+1, ((sp_der-sp_izq)/(ax1_ticks-1))))
ax12.xaxis.set_major_formatter(StrMethodFormatter('{x:.0f}'))
################################################################################

### RESD ###
res_izq=0.1     
res_der=100     

ax2.plot(well_train[crv_dp_res], well_train.index, color = "blue", linewidth = 2)
ax2.set_xlabel(f"Deep Resistivity ({well_train[crv_dp_res].name})")
ax2.set_xlim(res_izq, res_der)
ax2.xaxis.label.set_color("blue")
ax2.tick_params(axis='x', colors="blue")
ax2.spines["top"].set_edgecolor("blue")
ax2.set_xticks(np.arange(res_izq, res_der+.1, (res_der/(ax2_ticks-1))))
ax2.set_xscale("log")
ax2.grid(True, which="both", axis='x')
ax2.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### RESS ###


ax21.plot(well_train[crv_sh_res], well_train.index, color = "r", linewidth = 0.5)
ax21.set_xlabel(f"Shallow Resistivity ({well_train[crv_sh_res].name})")
ax21.set_xlim(res_izq, res_der)
ax21.xaxis.label.set_color("r")
ax21.spines["top"].set_position(("axes", 1.06))
ax21.spines["top"].set_visible(True)
ax21.tick_params(axis='x', colors="r")
ax21.spines["top"].set_edgecolor("r")
ax21.set_xticks(np.arange(res_izq, res_der+.1, ((res_der-res_izq)/(ax2_ticks-1))))
ax21.set_xscale("log")
ax21.xaxis.set_major_formatter(StrMethodFormatter('{x:.1f}'))
################################################################################

### DEN BULK ###
denbulk_izq = 1.65      
denbulk_der = 2.65      


ax3.plot(well_train[crv_den_bulk], well_train.index, color = "red", linewidth = 1.0)
ax3.set_xlabel(f"Density Bulk ({well_train[crv_den_bulk].name})")
ax3.set_xlim(denbulk_izq,denbulk_der)
ax3.xaxis.label.set_color("red")
ax3.tick_params(axis='x', colors="r")
ax3.spines["top"].set_edgecolor("r")
ax3.set_xticks(np.arange(denbulk_der, denbulk_izq+.01, ((denbulk_izq-denbulk_der)/(ax3_ticks-1))))
ax3.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### NEU POR ###
neupor_izq = .6     
neupor_der = 0      

ax31.plot(well_train[crv_neu_por], well_train.index, color = "blue", linewidth = 1.0)
ax31.set_xlabel(f"Neutron Porosity ({well_train[crv_neu_por].name})")
ax31.xaxis.label.set_color("b")
ax31.set_xlim(neupor_izq,neupor_der)
ax31.tick_params(axis='x', colors="blue")
ax31.spines["top"].set_position(("axes", 1.06))
ax31.spines["top"].set_visible(True)
ax31.spines["top"].set_edgecolor("blue")
ax31.set_xticks(np.arange(neupor_der, neupor_izq+.01, (abs(neupor_der-neupor_izq)/(ax3_ticks-1))))
ax31.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################

### AUX POR ###
auxpor_izq = .6    
auxpor_der = 0     
auxpor_desc = 'Empty Porosity'

ax32.plot(well_train[crv_nmr_por], well_train.index, color = "black", linewidth = 1.0)
ax32.set_xlabel(f"{auxpor_desc} ({well_train[crv_nmr_por].name})")
ax32.xaxis.label.set_color("black")
ax32.set_xlim(auxpor_izq,auxpor_der)
ax32.tick_params(axis='x', colors="black")
ax32.spines["top"].set_position(("axes", 1.1))
ax32.spines["top"].set_visible(True)
ax32.spines["top"].set_edgecolor("black")
ax32.set_xticks(np.arange(auxpor_der, auxpor_izq+.01, (abs(auxpor_der-auxpor_izq)/(ax3_ticks-1))))
ax32.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))
################################################################################
### TARGET & EMULATED ###

ax4.plot(well_train[crv_target], well_train.index, color = "black", linewidth = 1.5)
ax4.set_xlabel(f"Target ({well_train[crv_target].name})")
ax4.set_xlim(target_izq,target_der)
ax4.xaxis.label.set_color("black")
ax4.tick_params(axis='x', colors="black")
ax4.spines["top"].set_edgecolor("black")
ax4.set_xticks(np.arange(neupor_der, neupor_izq+.01, (abs(neupor_der-neupor_izq)/(ax4_ticks-1))))
ax4.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))


ax41.plot(well_train[crv_emulated], well_train.index, color = "red", linewidth = 0.5)
ax41.set_xlabel(f"Emulated ({well_train[crv_emulated].name})")
ax41.xaxis.label.set_color("r")
ax41.set_xlim(target_izq,target_der)
ax41.tick_params(axis='x', colors="red")
ax41.spines["top"].set_position(("axes", 1.06))
ax41.spines["top"].set_visible(True)
ax41.spines["top"].set_edgecolor("red")
ax41.set_xticks(np.arange(neupor_der, neupor_izq+.01, (abs(neupor_der-neupor_izq)/(ax4_ticks-1))))
ax41.xaxis.set_major_formatter(StrMethodFormatter('{x:.2f}'))



################################################################################


fig.suptitle("CURVA TARGET vs. CURVA EMULADA"+"\n"+"POZO: "+las.well.well.value,size=26,
             x=0.035,y=1,ha="left")

# Common functions for setting up the plot can be extracted into
# a for loop. This saves repeating code.

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylim(bot_plot, top_plot)
    ax.grid(which='major', color='lightgrey', linestyle='-')
    ax.xaxis.set_ticks_position("top")
    ax.xaxis.set_label_position("top")
    ax.spines["top"].set_position(("axes", 1.02))
 
    
for ax in [ax2, ax3, ax4]:
    plt.setp(ax.get_yticklabels(), visible = False)
  

fig.subplots_adjust(wspace = 0.2)

mplcursors.cursor(hover=True)
plt.tight_layout()
plt.show()